In [3]:
def prix_obligation(coupons, r, nominal, T):
    "r : taux d'actualisation "
    prix = 0
    for t in range(1,T+1):
        prix += coupons[t-1]/(1+r)**t
    prix += nominal/(1+r)**T
    return prix 

coupons = [5,5,5,5,5]
r = 0.03
nominal = 100
T = 5
prix = prix_obligation(coupons, r, nominal, T)
print(f"Le prix de l'obligation est {prix:.2f}")


Le prix de l'obligation est 109.16


Dans cet exemple le prix de l'obligation est de 109 € alors qu'on son prix au départ était de 100€ mais en réalité c'est normal puisque le taux d'actualisation est mtn a 3% alors que mon obligation est a 5% de rendement. 

In [4]:
def duration(coupons, r, T, prix):
    duree = 0
    for t in range(1,T+1):
        flux = coupons[t-1]
        if t ==T:
            flux+= nominal
        duree += (t*flux/(1+r)**t)
    duree = duree/prix
    sensibility = -duree/(1+r)
    return duree,sensibility

print(f"La duration de l'obligation est {duration(coupons, r, T, prix)[0]:.2f} années")
print(f"La sensibilité de l'obligation est {duration(coupons, r, T, prix)[1]:.4f}")

La duration de l'obligation est 4.57 années
La sensibilité de l'obligation est -4.4350


La sensibilité de l'obligation est de -4.4350, cela signifie que si le r varie de 1% sa valeur chute de 4.4350 %. Prix_apres_1% = 109 *((100-4.4350)/100) = 104.1


Si r passe de 0.03 à 0.07 alors ca fait 4 % de variation. Prix_apres_4% = 109(100-4*4.4350/100) = 91.6

In [6]:
def stress_test_portefeuille(portefeuille, choc_taux):
    print(f"{'Obligation':<15} | {'Prix Initial':<12} | {'Sensibilité':<12} | {'Perte (€)':<10}")
    print("-" * 60)
    
    perte_totale = 0
    valeur_totale_initiale = 0
    
    for nom, data in portefeuille.items():
        # Calcul du prix et de la sensibilité
        p = prix_obligation(data['coupons'], data['r'], data['nominal'], data['T'])
        dur, sens = duration(data['coupons'], data['r'], data['T'], p)
        
        # Calcul de la perte : Prix * Sensibilité * Choc
        perte_unitaire = p * sens * choc_taux
        
        perte_totale += perte_unitaire
        valeur_totale_initiale += p
        
        print(f"{nom:<15} | {p:<12.2f} | {sens:<12.4f} | {perte_unitaire:<10.2f}")

    print("-" * 60)
    print(f"VALEUR TOTALE DU BOOK : {valeur_totale_initiale:.2f} €")
    print(f"PERTE TOTALE EN CAS DE CHOC (+{choc_taux*100}%): {perte_totale:.2f} €")
    print(f"IMPACT RELATIF : {(perte_totale / valeur_totale_initiale)*100:.2f}%")


book = {
    "OAT 2 ans":    {'coupons': [1,1],   'r': 0.03, 'nominal': 100, 'T': 2},
    "OAT 10 ans":   {'coupons': [3]*10,  'r': 0.03, 'nominal': 100, 'T': 10},
    "OAT 30 ans":   {'coupons': [5]*30,  'r': 0.03, 'nominal': 100, 'T': 30}
}

stress_test_portefeuille(book, 0.02) # Choc de +2%

Obligation      | Prix Initial | Sensibilité  | Perte (€) 
------------------------------------------------------------
OAT 2 ans       | 96.17        | -1.9319      | -3.72     
OAT 10 ans      | 100.00       | -8.5302      | -17.06    
OAT 30 ans      | 139.20       | -17.7209     | -49.34    
------------------------------------------------------------
VALEUR TOTALE DU BOOK : 335.37 €
PERTE TOTALE EN CAS DE CHOC (+2.0%): -70.11 €
IMPACT RELATIF : -20.91%
